## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos
#### Autor: Jordy Carlos Carrión Loján
---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP4. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_cal.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_cal.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [3]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]
print("Cantidad de filas de df_creditos:", cantidad_filas_creditos)
print("Cantidad de filas de df_tarjetas:", cantidad_filas_tarjetas)

Cantidad de filas de df_creditos: 10127
Cantidad de filas de df_tarjetas: 10127


## Dimensión: completitud

### (1a) Filas

In [4]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna = df_creditos.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64


Este comando cuenta la cantidad total de valores nulos en el DataFrame df_creditos

In [5]:
df_creditos.isnull().sum(axis=1).sum()


1249

Este otro comando cuenta la cantidad de filas que contienen al menos un valor nulo en el DataFrame.

In [6]:
df_creditos.isnull().any(axis=1).sum()

1225

In [7]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print(f"Filas que incumplen el umbral de nulos en columnas [completitud_f]:")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f]:
0 (0.0)%


### (1b) Dataset

In [8]:
completitud_d = df_creditos.isnull().any(axis=1).sum()

print(f"Filas que presentan nulos en el dataset [completitud_d]:")
print(f"{completitud_d} ({round((completitud_d  / cantidad_filas_creditos) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d]:
1225 (12.1)%


----

## Dimensión: exactitud

### (2) Formato válido

In [9]:
# No se encuentran atributos con formato específico

----

## Dimensión: exactitud

### (3) Valores ajustados en dataset datos_creditos

Atributo: **edad**

In [10]:
df_creditos['edad'].isnull().sum()

0

In [11]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = pd.value_counts(df_creditos['edad']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['edad']) - df_creditos['edad'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
edad
22     2228
23     2190
24     1926
25     1646
26     1328
21      799
20        6
144       2
123       2
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
81,715623483.0,144,4800,3,4.0,ALQUILER,250000,INVERSIONES,0.02,13.57,0,N,0.000000
183,815500983.0,144,6000,2,4.0,HIPOTECA,200000,EDUCACIÓN,0.03,11.86,0,N,0.000000
575,717956283.0,123,20400,3,2.0,ALQUILER,80004,EDUCACIÓN,0.25,10.25,0,N,0.000000
747,712881483.0,123,20000,4,7.0,ALQUILER,78000,INVERSIONES,0.26,NaN,0,N,0.083333


Cantidad detectada: 4


In [12]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 4
Porcentaje de filas con errores de rango de valores (atributo edad): 0.04 %
Evaluación: no cumplimiento


Se procesa el atributo: **situacion_vivienda**


In [13]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = pd.value_counts(df_creditos['situacion_vivienda']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['situacion_vivienda_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['situacion_vivienda_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
situacion_vivienda
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [14]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


Se procesa el atributo: **objetivo_credito**

In [15]:
df_creditos['objetivo_credito'].unique()

array(['PERSONAL', 'EDUCACIÓN', 'SALUD', 'INVERSIONES', 'MEJORAS_HOGAR',
       'PAGO_DEUDAS'], dtype=object)

In [16]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: objetivo_credito

valores = pd.value_counts(df_creditos['objetivo_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'PERSONAL|EDUCACIÓN|SALUD|INVERSIONES|MEJORAS_HOGAR|PAGO_DEUDAS' # Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito_ok'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['objetivo_credito_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['objetivo_credito_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
objetivo_credito
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok


Cantidad detectada: 0


In [17]:
def calcular_rangos_valores_objetivo_credito():
    objetivo_credito_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo objetivo_credito: {objetivo_credito_valores_fuera_rango}")
    
    indicador = (objetivo_credito_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo objetivo_credito): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_objetivo_credito()

Cantidad de filas con valores fuera de rango en atributo objetivo_credito: 0
Porcentaje de filas con errores de rango de valores (atributo objetivo_credito): 0.0 %
Evaluación: ok


Se procesa el atributo: **estado_credito**

In [18]:
df_creditos['estado_credito'].unique()

array([1, 0], dtype=int64)

In [19]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_credito
# 0 El crédito aún no se ha cancelado.
# 1 El crédito se ha cancelado.

valores = pd.value_counts(df_creditos['estado_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['estado_credito']) - df_creditos['estado_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Se identifica y cuenta a los valores que son distintos de 0 o 1

resultado = df_creditos[(df_creditos['estado_credito'] != 0) & (df_creditos['estado_credito'] != 1)]

print("Se visualizan las filas con valores distintos de 0 o 1 en la columna 'estado_credito':")
display(resultado)  # Para visualizar las tuplas con valores distintos de 0 o 1

print(f"Cantidad de filas encontradas: {resultado.shape[0]}")


Distribución inicial del atributo: 
estado_credito
0    7635
1    2492
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con valores distintos de 0 o 1 en la columna 'estado_credito':


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok


Cantidad de filas encontradas: 0


In [20]:
def calcular_rangos_valores_estado_credito():
    estado_credito_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo estado_credito: {estado_credito_valores_fuera_rango}")
    
    indicador = (estado_credito_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (estado_credito): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_estado_credito()

Cantidad de filas con valores fuera de rango en atributo estado_credito: 0
Porcentaje de filas con errores de rango de valores (estado_credito): 0.0 %
Evaluación: ok


Se procesa el atributo: **falta_pago**

In [21]:
df_creditos['falta_pago'].unique()

array(['Y', 'N'], dtype=object)

In [22]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: falta_pago
# Y El crédito ha estado en mora en algún momento.
# N El crédito no ha entrado en mora.

valores = pd.value_counts(df_creditos['falta_pago']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['falta_pago']) - df_creditos['falta_pago'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'Y|N' # Se define una re de los valores validos 

df_creditos['falta_pago_ok'] = df_creditos['falta_pago'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[df_creditos['falta_pago_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['falta_pago_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
falta_pago
N    8359
Y    1768
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [23]:
def calcular_rangos_valores_falta_pago():
    falta_pago_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo falta_pago: {falta_pago_valores_fuera_rango}")
    
    indicador = (falta_pago_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo falta_pago): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_falta_pago()

Cantidad de filas con valores fuera de rango en atributo falta_pago: 0
Porcentaje de filas con errores de rango de valores (atributo falta_pago): 0.0 %
Evaluación: ok


Se procesa el atributo: **duracion_credito**

In [24]:
df_creditos['duracion_credito'].unique()

array([3, 2, 4], dtype=int64)

In [25]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: duracion_credito
# Tiempo en el que se distribuyen los pagos del crédito (en años). Por ejemplo, se establece que el tiempo máximo no debe superar los 10 años.
valores = pd.value_counts(df_creditos['duracion_credito']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['duracion_credito'] > 10]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
duracion_credito
2    3404
3    3364
4    3359
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok


Cantidad detectada: 0


In [26]:
def calcular_rangos_valores_duracion_credito():
    duracion_credito_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo duracion_credito: {duracion_credito_fuera_rango}")
    
    indicador = (duracion_credito_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo duracion_credito): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_duracion_credito()

Cantidad de filas con valores fuera de rango en atributo duracion_credito: 0
Porcentaje de filas con errores de rango de valores (atributo duracion_credito): 0.0 %
Evaluación: ok


## Dimensión: exactitud

### (3.1) Valores ajustados en dataset datos_tarjetas

Se procesa el atributo: **estado_civil**

In [27]:
df_tarjetas['estado_civil'].unique()

array(['CASADO', 'SOLTERO', 'DESCONOCIDO', 'DIVORCIADO'], dtype=object)

In [28]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_civil

valores = pd.value_counts(df_tarjetas['estado_civil']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_civil']) - df_tarjetas['estado_civil'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'CASADO|SOLTERO|DIVORCIADO|DESCONOCIDO' # Se define una re de los valores validos 

df_tarjetas['estado_civil_ok'] = df_tarjetas['estado_civil'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['estado_civil_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['estado_civil_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_civil
CASADO         4687
SOLTERO        3943
DESCONOCIDO     749
DIVORCIADO      748
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok


Cantidad detectada: 0


In [29]:
def calcular_rangos_valores_estado_civil():
    estado_civil_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo estado_civil: {estado_civil_valores_fuera_rango}")
    
    indicador = (estado_civil_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo estado_civil): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_estado_civil()

Cantidad de filas con valores fuera de rango en atributo estado_civil: 0
Porcentaje de filas con errores de rango de valores (atributo estado_civil): 0.0 %
Evaluación: ok


Se procesa el atributo: **estado_cliente**

In [30]:
df_tarjetas['estado_cliente'].unique()

array(['ACTIVO', 'PASIVO'], dtype=object)

In [31]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_cliente

valores = pd.value_counts(df_tarjetas['estado_cliente']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_cliente']) - df_tarjetas['estado_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ACTIVO|PASIVO' # Se define una re de los valores validos 

df_tarjetas['estado_cliente_ok'] = df_tarjetas['estado_cliente'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['estado_cliente_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['estado_cliente_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_cliente
ACTIVO    8500
PASIVO    1627
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok


Cantidad detectada: 0


In [32]:
def calcular_rangos_valores_estado_cliente():
    estado_cliente_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo estado_cliente: {estado_cliente_valores_fuera_rango}")
    
    indicador = (estado_cliente_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo estado_cliente): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_estado_cliente()

Cantidad de filas con valores fuera de rango en atributo estado_cliente: 0
Porcentaje de filas con errores de rango de valores (atributo estado_cliente): 0.0 %
Evaluación: ok


Se procesa el atributo: **nivel_tarjeta**

In [33]:
df_tarjetas['nivel_tarjeta'].unique()

array(['Blue', 'Gold', 'Silver', 'Platinum'], dtype=object)

In [34]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: nivel_tarjeta
'''
“Blue” El producto es del nivel básico.
“Gold” El producto es del nivel medio.
“Silver” El producto es del nivel avanzado.
“Platinum” El producto es del nivel más alto otorgado por la entidad.
'''

valores = pd.value_counts(df_tarjetas['nivel_tarjeta']) #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['nivel_tarjeta']) - df_tarjetas['nivel_tarjeta'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'Blue|Silver|Gold|Platinum' # Se define una re de los valores validos 

df_tarjetas['nivel_tarjeta_ok'] = df_tarjetas['nivel_tarjeta'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[df_tarjetas['nivel_tarjeta_ok'] == False]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['nivel_tarjeta_ok'] == False] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
nivel_tarjeta
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok


Cantidad detectada: 0


In [35]:
def calcular_rangos_valores_nivel_tarjeta():
    nivel_tarjeta_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo nivel_tarjeta: {nivel_tarjeta_valores_fuera_rango}")
    
    indicador = (nivel_tarjeta_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo nivel_tarjeta): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_nivel_tarjeta()

Cantidad de filas con valores fuera de rango en atributo nivel_tarjeta: 0
Porcentaje de filas con errores de rango de valores (atributo nivel_tarjeta): 0.0 %
Evaluación: ok


---

## Dimensión: consistencia

### (4) Claves únicas

Dataset: **datos_creditos**

In [36]:
#Se obtiene el valor de la cantidad de filas actual
cant_antes = df_creditos.shape[0] 

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_creditos.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_creditos.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_creditos.shape[0]

# Se imprimen ambos valores
print('Dataset: creditos')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: creditos
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


Dataset: **datos_tarjetas**

In [37]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [38]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,True,True,True
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,992.0,M,3421.0,DESCONOCIDO,Blue,21.0,0.0,True,True,True
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,1447.0,M,8258.0,POSGRADO_COMPLETO,Blue,23.0,3.0,True,True,True
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,Blue,82.0,2.0,True,True,True
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,Blue,59.0,2.0,True,True,True


Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [39]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

In [40]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

indicador = (aux.shape[0] / cantidad_filas_creditos)
print(f"Porcentaje de filas con incumplimiento de la Regla 1: {round(indicador * 100, 2)} %")


Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok,regla_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,M,19802.0,UNIVERSITARIO_INCOMPLETO,Blue,58.0,4.0,True,True,True,err
1550,711232608.0,21,8000,4,3.0,ALQUILER,15500,MEJORAS_HOGAR,0.52,7.51,...,F,1636.0,SECUNDARIO_COMPLETO,Blue,35.0,5.0,True,True,True,err
1900,711864783.0,24,10000,2,0.0,HIPOTECA,18000,PAGO_DEUDAS,0.56,12.18,...,M,2201.0,UNIVERSITARIO_INCOMPLETO,Blue,32.0,1.0,True,True,True,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,F,1684.0,UNIVERSITARIO_INCOMPLETO,Blue,45.0,3.0,True,True,True,err
2809,713555133.0,22,8000,4,3.0,ALQUILER,14400,MEJORAS_HOGAR,0.56,9.99,...,F,1667.0,SECUNDARIO_COMPLETO,Blue,40.0,3.0,True,True,True,err


Cantidad de filas que no cumplen la regla: 15
Porcentaje de filas con incumplimiento de la Regla 1: 0.15 %


Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [42]:
# Se define una función para aplicar los cálculos de la Regla 2
def regla_duracion_credito(row):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda
    
    if duracion_credito == df_integrado['duracion_credito'].min() and pct_ingreso > 0.6 and situacion_vivienda != 'PROPIA':
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_credito_resultado = df_integrado.apply(lambda row: regla_duracion_credito(row), axis=1).rename("regla_duracion_credito")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_duracion_credito_resultado], axis=1)  

# Se visualizan los datos que no cumplen con la regla
print("Se visualizan las tuplas que no cumplen con la Regla 2:\n")
display(df_resultado[df_resultado.regla_duracion_credito == 'err'].head())

# Se verifica la cantidad de elementos que no cumplen la regla
aux_regla_duracion_credito = df_resultado[df_resultado.regla_duracion_credito == 'err']
print(f"Cantidad de filas que no cumplen la Regla 2: {aux_regla_duracion_credito.shape[0]}")

# Se calcula el porcentaje de filas que no cumplen la regla respecto al total de créditos
porcentaje_regla_duracion_credito = (aux_regla_duracion_credito.shape[0] / df_integrado.shape[0]) * 100
print(f"Porcentaje de filas con incumplimiento de la Regla 2: {round(porcentaje_regla_duracion_credito, 2)} %")


Se visualizan las tuplas que no cumplen con la Regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok,regla_duracion_credito
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,M,19802.0,UNIVERSITARIO_INCOMPLETO,Blue,58.0,4.0,True,True,True,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,M,8714.0,UNIVERSITARIO_INCOMPLETO,Blue,38.0,4.0,True,True,True,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,F,12373.0,UNIVERSITARIO_COMPLETO,Blue,40.0,2.0,True,True,True,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,F,1684.0,UNIVERSITARIO_INCOMPLETO,Blue,45.0,3.0,True,True,True,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,F,3035.0,UNIVERSITARIO_COMPLETO,Blue,37.0,3.0,True,True,True,err


Cantidad de filas que no cumplen la Regla 2: 7
Porcentaje de filas con incumplimiento de la Regla 2: 0.07 %


In [43]:
df_integrado.head()

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,True,True,True
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,992.0,M,3421.0,DESCONOCIDO,Blue,21.0,0.0,True,True,True
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,1447.0,M,8258.0,POSGRADO_COMPLETO,Blue,23.0,3.0,True,True,True
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,Blue,82.0,2.0,True,True,True
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,Blue,59.0,2.0,True,True,True


In [44]:
df_creditos.head()

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok
9818,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,True,True
1590,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,0.0,True,True,True
4419,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,0.0,True,True,True
6148,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,0.0,True,True,True
6784,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,0.0,True,True,True


In [45]:
df_tarjetas.head()

,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,estado_civil_ok,estado_cliente_ok,nivel_tarjeta_ok
9818,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,True,True,True
1590,708083283.0,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,Blue,21.0,0.0,True,True,True
4419,708084558.0,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,Blue,23.0,3.0,True,True,True
6148,708085458.0,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,Blue,82.0,2.0,True,True,True
6784,708086958.0,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,Blue,59.0,2.0,True,True,True


In [46]:
df_creditos.to_csv("../../data/processed/datos_creditos_mc.csv", sep=";", index=False)

In [47]:
df_tarjetas.to_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";", index=False)
df_resultado.to_csv("../../data/processed/datos_integrados.csv", sep=";", index=False)